# Homework 1: Stance Classification

* 建立一个分类器，根据一段文字，预测pro vs. con
* 仅使用**主题'abortion'和'gay rights'**
* 在csv中，每行一个帖子。columns:帖子和信息，6个LIWC1特征，以及句子的3个词性标签的数量
* 用5-fold CV来评估模型，选择最佳features和每个主题的模型类型。 报告平均准确度和F1分，每个主题的前20个功能。
* 提交必须包含README文件
* 试验SVM和naive bayes
* Input: features. Output: label $y \in\{0,1\}$, “pro” (1) or “con” (0).

**每一个topic用两个模型**:
1. Ngrams: combination of unigrams, bigrams and trigrams as features.(实验找到最好的)
2. 对上面每个features set, 选模型SVM/NB. 也需要调参数防止overfitting (scikit-learn有包，Some examples are “kernel” and “C” for SVM.)

**Feature selection**
每组第二个模型可以考虑以下的feature或者额外的：
* Ngrams. Unigrams, bigrams, and trigrams.
* Cue words. The initial unigram, bigram, and trigram in a post.
* Repeated punctuation. Features to capture punctuation such as ??, ??????, !!!, or ?!.
* Part-of-speech tags. Features to count the number of nouns, verbs and adjectives in the
text (count_noun, count_verb, and count_adj).
* LIWC features. We give you a number of LIWC features in the data files. You may use
any combination of these as features.The measures we give and their descriptions are:
  * word_count: the number of words in the post.
  * words_pronom: the percent of words in the text that are pronominal.
  * words_per_sen: the average number of words per sentence in the text.
  * words_over_6: the percent of words in the text that are over 6 letters long.
  * pos_emo: the number of positive emotion words in the text.
  * neg_emo: the number of negative emotion words in the text.
* Any other features you come up with.

### 提交要求：
* 对于每个主题，请提供您最佳模型的书面说明，并提供合理的理由您使用或未使用的功能。您为什么认为他们有帮助？您
必须明确列出您在此处尝试过的功能。
* 对每个主题进行错误分析。为了获得最佳表现模型，请选择3个有趣的例子不起作用。指出为什么您认为它们的分类不正确。如果你可以添加其他功能，您认为哪些功能可以帮助提高性能？
对于每个示例，如果篇幅很长，则无需包括整个帖子文本，只需将您的讨论的相关部分和ID。
* 比较和对比您用于每个主题的功能。为什么你会想到一些
功能在不同主题下效果更好还是更差？还是您将相同的功能用于
所有主题，如果可以，为什么您认为效果很好？


### 评分
您将按照以下要素进行评分：
#### 2.1分类器（共60分）
您的交付品如下（50分）：
* 一个将火车文件和主题作为命令行参数的classify.py文件（即，可以作为“ python classify.py train-data.csv中止”运行），运行交叉验证选择模型，打印模型的平均精度和F1度量并打印顶部该主题的20个功能。 （60分）。
* 您的自述文件，如下所述（2.3）
* 准确性（10分）
您的系统将获得最高10分，具体取决于最佳模型是否产生了准确性高于或低于我们的阈值。
这些将在9月13日星期五之前发布在Piazza和课程网站上。

#### 2.2书面答案（30分）
* 为每个主题提供最佳性能模型的依据：说明完整且提供
有关每个模型（及其使用或不使用的功能）为何最好的信息。 （20分）
* 错误分析：所选职位应有所不同，说明应明确说明为什么功能无法预测正确的类别。 （10点）
您的可交付成果如下：
包含您的姓名，电子邮件地址，作业编号和
您对书面部分的回答。
* 2.3软件工程（包括文档）（10分）
您的自述文件必须包含以下内容：
您的姓名和电子邮件地址。
作业编号。
有关如何训练和测试分类器的信息。
分类器的特殊功能（或限制）的描述。
在代码文档中：
应该以有意义的方式记录代码。这可以表示表达功能/变量
名称以及评论。
信息性的方法/功能/变量名称。
高效实施。


### 提交说明
您应该在Courseworks上提交以下内容：
一个名为<YOUR-UNI> -hw1.zip的zip文件。这应该确切地列出了文件第2.1节中的可交付成果。您无需在zip中包含数据文件。
您应该在Gradescope上提交以下内容：
如第2.2节所述的hw1-write.pdf。

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
import numpy as np
import nltk
import pandas as pd

data = pd.read_csv('/content/gdrive/My Drive/NLP/stance-data.csv')
data.head()

,post_text,topic,author,label,id,word_count,words_pronom,words_per_sen,words_over_6,pos_emo,neg_emo,count_noun,count_verb,count_adj
0,There is only one question when it comes to ab...,abortion,jstantall,con,C0,185,18.50,1.62,3.78,16.22,12.97,41,33,15
1,People often try to personalize their pro-life...,abortion,NVYN,pro,C1,85,10.62,3.53,1.18,11.76,21.18,12,19,2
2,we 're saying that the pregnant person should ...,abortion,atypican,pro,C2,41,13.67,2.44,2.44,7.32,17.07,8,10,3
3,"No answer to my retort , just a down vote . Th...",abortion,atypican,pro,C3,12,6.00,8.33,0.00,0.00,16.67,3,2,1
4,I 'm sooo sick of people like you who feel lik...,abortion,SMCdeBater,con,C4,34,17.00,0.00,8.82,14.71,23.53,5,10,2


In [0]:
#np.shape(data)

(4649, 14)

In [0]:
#set(data.topic)

{'abortion', 'gay rights', 'marijuana', 'obama'}

# 1. Test methods to find best model with topic == 'abortion'

## 1.2. Test with Ngrams (GridSearchCV)

In [0]:
topic = "abortion"
test = data[data.topic==topic][["post_text","label"]]
#test.head()

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import BernoulliNB, MultinomialNB, ComplementNB
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.svm import LinearSVC
text_clf = Pipeline([
    ('vect', CountVectorizer(stop_words='english')),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])
from sklearn.model_selection import GridSearchCV
parameters = {
    'vect__ngram_range':[(1,1),(1,2),(1,3),(1,4)],
    'tfidf__use_idf':(True,False),
    'clf__alpha':(1e-1,1e-2,1e-3,1e-4),
}

In [0]:
gs_clf = GridSearchCV(text_clf, parameters, cv=5, iid=False, n_jobs=-1)
gs_clf = gs_clf.fit(test.post_text,test.label)
print(gs_clf.best_score_)
for param_name in sorted(parameters.keys()):
  print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))

0.5022362847490183
clf__alpha: 0.1
tfidf__use_idf: False
vect__ngram_range: (1, 1)


topic="abortion", grid search with 'vect__ngram_range':[(1,1),(1,2),(1,3),(1,4)],
    'tfidf__use_idf':(True,False),
    'clf__alpha':(1e-1,1e-2,1e-3,1e-4), MultinomialNB().
    
Get best scores 0.5022362847490183 with
clf__alpha: 0.1, 
tfidf__use_idf: False, 
vect__ngram_range: (1, 1).

## Add clf__alpha = 0.1, best 0.5 and clf__alpha, even lower. Check whether it is due to the topic

In [0]:
topic = "gay rights"
test2 = data[data.topic==topic][["post_text","label"]]

In [0]:
gs_clf = GridSearchCV(text_clf, parameters, cv=5, iid=False, n_jobs=-1)
gs_clf = gs_clf.fit(test2.post_text,test2.label)
print(gs_clf.best_score_)
for param_name in sorted(parameters.keys()):
  print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))

0.6011191760269251
clf__alpha: 0.1
tfidf__use_idf: False
vect__ngram_range: (1, 1)


topic=gay rights, grid search with same parameter ranges: [(1,1),(1,2),(1,3),(1,4)], 'tfidf__use_idf':(True,False), 'clf__alpha':(1e-1,1e-2,1e-3,1e-4), MultinomialNB().

Get best scores 0.6011191760269251 with clf__alpha: 0.1,
tfidf__use_idf: False,
vect__ngram_range: (1, 1),

### 尝试是否是MultinomialNB中clf_alpha的取值影响

In [0]:
text_clf = Pipeline([
    ('vect', CountVectorizer(stop_words='english')),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])
from sklearn.model_selection import GridSearchCV
parameters = {
    'vect__ngram_range':[(1,1),(1,2),(1,3),(1,4)],
    'tfidf__use_idf':(True,False),
    'clf__alpha':(10, 1,1e-1,1e-2,1e-3),
}

In [0]:
gs_clf = GridSearchCV(text_clf, parameters, cv=5, iid=False, n_jobs=-1)
gs_clf = gs_clf.fit(test.post_text,test.label)
print(gs_clf.best_score_)
for param_name in sorted(parameters.keys()):
  print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))

0.553279827133817
clf__alpha: 10
tfidf__use_idf: True
vect__ngram_range: (1, 2)


In [0]:
text_clf = Pipeline([
    ('vect', CountVectorizer(stop_words='english')),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])
from sklearn.model_selection import GridSearchCV
parameters = {
    'vect__ngram_range':[(1,1),(1,2),(1,3),(1,4)],
    'tfidf__use_idf':(True,False),
    'clf__alpha':(10,7,4, 1),
}
gs_clf = GridSearchCV(text_clf, parameters, cv=5, iid=False, n_jobs=-1)
gs_clf = gs_clf.fit(test.post_text,test.label)
print(gs_clf.best_score_)
for param_name in sorted(parameters.keys()):
  print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))

0.553279827133817
clf__alpha: 10
tfidf__use_idf: True
vect__ngram_range: (1, 2)


In [0]:
text_clf = Pipeline([
    ('vect', CountVectorizer(stop_words='english')),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])
from sklearn.model_selection import GridSearchCV
parameters = {
    'vect__ngram_range':[(1,1),(1,2),(1,3),(1,4)],
    'tfidf__use_idf':(True,False),
    'clf__alpha':(2,1,0.5,1e-1),
}
gs_clf = GridSearchCV(text_clf, parameters, cv=5, iid=False, n_jobs=-1)
gs_clf = gs_clf.fit(test.post_text,test.label)
print(gs_clf.best_score_)
for param_name in sorted(parameters.keys()):
  print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))

0.5415427620860558
clf__alpha: 2
tfidf__use_idf: False
vect__ngram_range: (1, 2)


In [0]:
gs_clf = GridSearchCV(text_clf, parameters, cv=5, iid=False, n_jobs=-1)
gs_clf = gs_clf.fit(test2.post_text,test2.label)
print(gs_clf.best_score_)
for param_name in sorted(parameters.keys()):
  print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))

0.6416501088087804
clf__alpha: 10
tfidf__use_idf: True
vect__ngram_range: (1, 1)


## Try different classification

#### 1.MultinomialNB
**In https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html, write the one most suitable for word counts is the multinomial variant.**

**Paramter alpha in MultinomialNB is smoothing variable**. $\alpha \ge 0$ accounts for features not present in the learning samples and prevents zero probabilities in further computations. Setting $\alpha=1$ is called Laplace smoothing, while $\alpha <1$ is called Lidstone smoothing.

#### 2. ComplementNB
ComplementNB是MultinomialNB升级版，更适合imbalanced data, 而且常常结果更好。

#### 3. BernoulliNB
在某些dataset很好，特别是文档很短的时候。

#### 最终建议 是每种都试下。。。

In [0]:
text_clf = Pipeline([
    ('vect', CountVectorizer(stop_words='english')),
    ('tfidf', TfidfTransformer()),
    ('clf', ComplementNB()),
])
from sklearn.model_selection import GridSearchCV
parameters = {
    'vect__ngram_range':[(1,1),(1,2),(1,3),(1,4)],
    'tfidf__use_idf':(True,False),
    'clf__alpha':(1,0.5,1e-1,1e-2,1e-3),
}
gs_clf = GridSearchCV(text_clf, parameters, cv=5, iid=False, n_jobs=-1)
gs_clf = gs_clf.fit(test.post_text,test.label)
print(gs_clf.best_score_)
for param_name in sorted(parameters.keys()):
  print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))

0.527457941040284
clf__alpha: 1
tfidf__use_idf: False
vect__ngram_range: (1, 1)


In [0]:
text_clf = Pipeline([
    ('vect', CountVectorizer(stop_words='english')),
    ('tfidf', TfidfTransformer()),
    ('clf', BernoulliNB()),
])
from sklearn.model_selection import GridSearchCV
parameters = {
    'vect__ngram_range':[(1,1),(1,2),(1,3),(1,4)],
    'tfidf__use_idf':(True,False),
    'clf__alpha':(1,0.5,1e-1,1e-2,1e-3),
}
gs_clf = GridSearchCV(text_clf, parameters, cv=5, iid=False, n_jobs=-1)
gs_clf = gs_clf.fit(test.post_text,test.label)
print(gs_clf.best_score_)
for param_name in sorted(parameters.keys()):
  print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))

0.5075749001046115
clf__alpha: 1
tfidf__use_idf: True
vect__ngram_range: (1, 4)


In [0]:
gs_clf.score

## SVM classifier uses SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, random_state=42,max_iter=5, tol=None) ##会比NB更好，但是慢点

In [0]:
from sklearn.linear_model import SGDClassifier
text_clf_SVM = Pipeline([
    ('vect', CountVectorizer(stop_words='english')),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier()),
])
from sklearn.model_selection import GridSearchCV
parameters_SVM = {
    'vect__ngram_range':[(1,1),(1,2),(1,3)],
    'tfidf__use_idf':(True,False),
    'clf__alpha':(1,1e-1,1e-2,1e-3),
    'clf__penalty':('l1','l2'),
}
gs_clf_SVM = GridSearchCV(text_clf_SVM, parameters_SVM, cv=5, iid=False, n_jobs=-1)
gs_clf_SVM = gs_clf_SVM.fit(test.post_text,test.label)
print(gs_clf_SVM.best_score_)
for param_name in sorted(parameters_SVM.keys()):
  print("%s: %r" % (param_name, gs_clf_SVM.best_params_[param_name]))

0.5877844660527173
clf__alpha: 0.001
clf__penalty: 'l1'
tfidf__use_idf: False
vect__ngram_range: (1, 2)


In [0]:
gs_clf_SVM = GridSearchCV(text_clf_SVM, parameters_SVM, cv=5, iid=False, n_jobs=-1)
gs_clf_SVM = gs_clf_SVM.fit(test2.post_text,test2.label)
print(gs_clf_SVM.best_score_)
for param_name in sorted(parameters_SVM.keys()):
  print("%s: %r" % (param_name, gs_clf_SVM.best_params_[param_name]))

0.6416501088087804
clf__alpha: 1
clf__penalty: 'l1'
tfidf__use_idf: True
vect__ngram_range: (1, 1)


## 1.3. Test with more features (GridSearchCV)

In [8]:
shuffled_test[:,1]

array(['abortion', 'abortion', 'abortion', ..., 'abortion', 'abortion',
       'abortion'], dtype=object)

###!只有6个liwc features

In [0]:
topic = "abortion"
test = data[data.topic==topic][["post_text","label","word_count","words_pronom",
                                      "words_per_sen","words_over_6","pos_emo","neg_emo"]]
shuffled_test = np.random.permutation(test)
cv = CountVectorizer(stop_words='english', ngram_range=(1,2))
test_cv = cv.fit_transform(shuffled_test[:,0])
from scipy.sparse import hstack
test_X = hstack((test_cv, shuffled_test[:,2:].astype(float)))
test_y = shuffled_test[:,1]

#### using NB-best: MultinomialNB

In [13]:
text_clf = Pipeline([
    ('clf', MultinomialNB()),
])
from sklearn.model_selection import GridSearchCV
parameters = {
    'clf__alpha':(1,1e-1,1e-2,1e-3),
}
gs_clf = GridSearchCV(text_clf, parameters, cv=5, iid=False, n_jobs=-1)
gs_clf = gs_clf.fit(test_X, test_y)
print(gs_clf.best_score_)
for param_name in sorted(parameters.keys()):
  print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))

0.553254102999434
clf__alpha: 1


#### using svm

In [14]:
cv = CountVectorizer(stop_words='english', ngram_range=(1,1))
test_cv = cv.fit_transform(shuffled_test[:,0])
from scipy.sparse import hstack
test_X = hstack((test_cv, shuffled_test[:,2:].astype(float)))
test_y = shuffled_test[:,1]

from sklearn.linear_model import SGDClassifier
text_clf_SVM = Pipeline([
    ('clf', SGDClassifier()),
])
from sklearn.model_selection import GridSearchCV
parameters_SVM = {
    'clf__alpha':(1,1e-1,1e-2,1e-3),
    'clf__penalty':('l1','l2'),
}
gs_clf_SVM = GridSearchCV(text_clf_SVM, parameters_SVM, cv=5, iid=False, n_jobs=-1)
gs_clf_SVM = gs_clf_SVM.fit(test_X, test_y)
print(gs_clf_SVM.best_score_)
for param_name in sorted(parameters_SVM.keys()):
  print("%s: %r" % (param_name, gs_clf_SVM.best_params_[param_name]))

0.5854692939582583
clf__alpha: 0.001
clf__penalty: 'l1'


In [15]:
cv = CountVectorizer(stop_words='english', ngram_range=(1,2))
test_cv = cv.fit_transform(shuffled_test[:,0])
from scipy.sparse import hstack
test_X = hstack((test_cv, shuffled_test[:,2:].astype(float)))
test_y = shuffled_test[:,1]

from sklearn.linear_model import SGDClassifier
text_clf_SVM = Pipeline([
    ('clf', SGDClassifier()),
])
from sklearn.model_selection import GridSearchCV
parameters_SVM = {
    'clf__alpha':(1,1e-1,1e-2,1e-3),
    'clf__penalty':('l1','l2'),
}
gs_clf_SVM = GridSearchCV(text_clf_SVM, parameters_SVM, cv=5, iid=False, n_jobs=-1)
gs_clf_SVM = gs_clf_SVM.fit(test_X, test_y)
print(gs_clf_SVM.best_score_)
for param_name in sorted(parameters_SVM.keys()):
  print("%s: %r" % (param_name, gs_clf_SVM.best_params_[param_name]))

0.5678791308672463
clf__alpha: 0.01
clf__penalty: 'l2'


In [19]:
cv = CountVectorizer(stop_words='english', ngram_range=(1,1))
test_cv = cv.fit_transform(shuffled_test[:,0])
from scipy.sparse import hstack
test_X = hstack((test_cv, shuffled_test[:,2:3].astype(float)))
test_y = shuffled_test[:,1]

from sklearn.linear_model import SGDClassifier
text_clf_SVM = Pipeline([
    ('clf', SGDClassifier()),
])
from sklearn.model_selection import GridSearchCV
parameters_SVM = {
    'clf__alpha':(1,1e-1,1e-2,1e-3),
    'clf__penalty':('l1','l2'),
}
gs_clf_SVM = GridSearchCV(text_clf_SVM, parameters_SVM, cv=5, iid=False, n_jobs=-1)
gs_clf_SVM = gs_clf_SVM.fit(test_X, test_y)
print(gs_clf_SVM.best_score_)
for param_name in sorted(parameters_SVM.keys()):
  print("%s: %r" % (param_name, gs_clf_SVM.best_params_[param_name]))

0.57727872957075
clf__alpha: 0.01
clf__penalty: 'l1'


In [20]:
cv = CountVectorizer(stop_words='english', ngram_range=(1,1))
test_cv = cv.fit_transform(shuffled_test[:,0])
from scipy.sparse import hstack
test_X = hstack((test_cv, shuffled_test[:,2:4].astype(float)))
test_y = shuffled_test[:,1]

from sklearn.linear_model import SGDClassifier
text_clf_SVM = Pipeline([
    ('clf', SGDClassifier()),
])
from sklearn.model_selection import GridSearchCV
parameters_SVM = {
    'clf__alpha':(1,1e-1,1e-2,1e-3),
    'clf__penalty':('l1','l2'),
}
gs_clf_SVM = GridSearchCV(text_clf_SVM, parameters_SVM, cv=5, iid=False, n_jobs=-1)
gs_clf_SVM = gs_clf_SVM.fit(test_X, test_y)
print(gs_clf_SVM.best_score_)
for param_name in sorted(parameters_SVM.keys()):
  print("%s: %r" % (param_name, gs_clf_SVM.best_params_[param_name]))

0.5854778686697192
clf__alpha: 0.001
clf__penalty: 'l1'


In [21]:
cv = CountVectorizer(stop_words='english', ngram_range=(1,1))
test_cv = cv.fit_transform(shuffled_test[:,0])
from scipy.sparse import hstack
test_X = hstack((test_cv, shuffled_test[:,2:5].astype(float)))
test_y = shuffled_test[:,1]

from sklearn.linear_model import SGDClassifier
text_clf_SVM = Pipeline([
    ('clf', SGDClassifier()),
])
from sklearn.model_selection import GridSearchCV
parameters_SVM = {
    'clf__alpha':(1,1e-1,1e-2,1e-3),
    'clf__penalty':('l1','l2'),
}
gs_clf_SVM = GridSearchCV(text_clf_SVM, parameters_SVM, cv=5, iid=False, n_jobs=-1)
gs_clf_SVM = gs_clf_SVM.fit(test_X, test_y)
print(gs_clf_SVM.best_score_)
for param_name in sorted(parameters_SVM.keys()):
  print("%s: %r" % (param_name, gs_clf_SVM.best_params_[param_name]))

0.5661487540944248
clf__alpha: 0.01
clf__penalty: 'l2'


In [22]:
cv = CountVectorizer(stop_words='english', ngram_range=(1,1))
test_cv = cv.fit_transform(shuffled_test[:,0])
from scipy.sparse import hstack
test_X = hstack((test_cv, shuffled_test[:,2:6].astype(float)))
test_y = shuffled_test[:,1]

from sklearn.linear_model import SGDClassifier
text_clf_SVM = Pipeline([
    ('clf', SGDClassifier()),
])
from sklearn.model_selection import GridSearchCV
parameters_SVM = {
    'clf__alpha':(1,1e-1,1e-2,1e-3),
    'clf__penalty':('l1','l2'),
}
gs_clf_SVM = GridSearchCV(text_clf_SVM, parameters_SVM, cv=5, iid=False, n_jobs=-1)
gs_clf_SVM = gs_clf_SVM.fit(test_X, test_y)
print(gs_clf_SVM.best_score_)
for param_name in sorted(parameters_SVM.keys()):
  print("%s: %r" % (param_name, gs_clf_SVM.best_params_[param_name]))

0.5749481229956612
clf__alpha: 1
clf__penalty: 'l2'


In [23]:
cv = CountVectorizer(stop_words='english', ngram_range=(1,1))
test_cv = cv.fit_transform(shuffled_test[:,0])
from scipy.sparse import hstack
test_X = hstack((test_cv, shuffled_test[:,3:4].astype(float)))
test_y = shuffled_test[:,1]

from sklearn.linear_model import SGDClassifier
text_clf_SVM = Pipeline([
    ('clf', SGDClassifier()),
])
from sklearn.model_selection import GridSearchCV
parameters_SVM = {
    'clf__alpha':(1,1e-1,1e-2,1e-3),
    'clf__penalty':('l1','l2'),
}
gs_clf_SVM = GridSearchCV(text_clf_SVM, parameters_SVM, cv=5, iid=False, n_jobs=-1)
gs_clf_SVM = gs_clf_SVM.fit(test_X, test_y)
print(gs_clf_SVM.best_score_)
for param_name in sorted(parameters_SVM.keys()):
  print("%s: %r" % (param_name, gs_clf_SVM.best_params_[param_name]))

0.591320677059217
clf__alpha: 0.01
clf__penalty: 'l1'


In [24]:
cv = CountVectorizer(stop_words='english', ngram_range=(1,1))
test_cv = cv.fit_transform(shuffled_test[:,0])
from scipy.sparse import hstack
test_X = hstack((test_cv, shuffled_test[:,3:].astype(float)))
test_y = shuffled_test[:,1]

from sklearn.linear_model import SGDClassifier
text_clf_SVM = Pipeline([
    ('clf', SGDClassifier()),
])
from sklearn.model_selection import GridSearchCV
parameters_SVM = {
    'clf__alpha':(1,1e-1,1e-2,1e-3),
    'clf__penalty':('l1','l2'),
}
gs_clf_SVM = GridSearchCV(text_clf_SVM, parameters_SVM, cv=5, iid=False, n_jobs=-1)
gs_clf_SVM = gs_clf_SVM.fit(test_X, test_y)
print(gs_clf_SVM.best_score_)
for param_name in sorted(parameters_SVM.keys()):
  print("%s: %r" % (param_name, gs_clf_SVM.best_params_[param_name]))

0.5942583732057416
clf__alpha: 0.001
clf__penalty: 'l1'


In [25]:
cv = CountVectorizer(stop_words='english', ngram_range=(1,1))
test_cv = cv.fit_transform(shuffled_test[:,0])
from scipy.sparse import hstack
test_X = hstack((test_cv, shuffled_test[:,3:5].astype(float)))
test_y = shuffled_test[:,1]

from sklearn.linear_model import SGDClassifier
text_clf_SVM = Pipeline([
    ('clf', SGDClassifier()),
])
from sklearn.model_selection import GridSearchCV
parameters_SVM = {
    'clf__alpha':(1,1e-1,1e-2,1e-3),
    'clf__penalty':('l1','l2'),
}
gs_clf_SVM = GridSearchCV(text_clf_SVM, parameters_SVM, cv=5, iid=False, n_jobs=-1)
gs_clf_SVM = gs_clf_SVM.fit(test_X, test_y)
print(gs_clf_SVM.best_score_)
for param_name in sorted(parameters_SVM.keys()):
  print("%s: %r" % (param_name, gs_clf_SVM.best_params_[param_name]))

0.5925074171254137
clf__alpha: 0.01
clf__penalty: 'l2'


In [26]:
cv = CountVectorizer(stop_words='english', ngram_range=(1,1))
test_cv = cv.fit_transform(shuffled_test[:,0])
from scipy.sparse import hstack
test_X = hstack((test_cv, shuffled_test[:,3:6].astype(float)))
test_y = shuffled_test[:,1]

from sklearn.linear_model import SGDClassifier
text_clf_SVM = Pipeline([
    ('clf', SGDClassifier()),
])
from sklearn.model_selection import GridSearchCV
parameters_SVM = {
    'clf__alpha':(1,1e-1,1e-2,1e-3),
    'clf__penalty':('l1','l2'),
}
gs_clf_SVM = GridSearchCV(text_clf_SVM, parameters_SVM, cv=5, iid=False, n_jobs=-1)
gs_clf_SVM = gs_clf_SVM.fit(test_X, test_y)
print(gs_clf_SVM.best_score_)
for param_name in sorted(parameters_SVM.keys()):
  print("%s: %r" % (param_name, gs_clf_SVM.best_params_[param_name]))

0.6024300732280359
clf__alpha: 0.1
clf__penalty: 'l2'


In [33]:
cv = CountVectorizer(stop_words='english', ngram_range=(1,1))
test_cv = cv.fit_transform(shuffled_test[:,0])
from scipy.sparse import hstack
test_X = hstack((test_cv, shuffled_test[:,3:6].astype(float)))
test_y = shuffled_test[:,1]

from sklearn.linear_model import SGDClassifier
text_clf_SVM = Pipeline([
    ('clf', SGDClassifier()),
])
from sklearn.model_selection import GridSearchCV
parameters_SVM = {
    'clf__alpha':(1,1e-1,1e-2,1e-3),
    'clf__penalty':('l1','l2','None'),
    'clf__loss':('hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'),
    'clf__max_iter':(1000,5000,10000,20000),
    
}
gs_clf_SVM = GridSearchCV(text_clf_SVM, parameters_SVM, cv=5, iid=False, n_jobs=-1)
gs_clf_SVM = gs_clf_SVM.fit(test_X, test_y)
print(gs_clf_SVM.best_score_)
for param_name in sorted(parameters_SVM.keys()):
  print("%s: %r" % (param_name, gs_clf_SVM.best_params_[param_name]))

0.6118039477971566
clf__alpha: 1
clf__loss: 'modified_huber'
clf__max_iter: 1000
clf__penalty: 'l2'


In [42]:
from sklearn.svm import LinearSVC
text_clf_SVM = Pipeline([
    ('clf', LinearSVC()),
])
from sklearn.model_selection import GridSearchCV
parameters_SVM = {
    #'clf__penalty':('l1','l2'),
    'clf__dual':(True, False),
    #'clf__loss':('hinge', 'squared_hinge'),
    'clf__C':(1, 0.5,0.1),
    
}
gs_clf_SVM = GridSearchCV(text_clf_SVM, parameters_SVM, cv=5, n_jobs=-1)
gs_clf_SVM = gs_clf_SVM.fit(test_X, test_y)
print(gs_clf_SVM.best_score_)
for param_name in sorted(parameters_SVM.keys()):
  print("%s: %r" % (param_name, gs_clf_SVM.best_params_[param_name]))

0.5954332552693209
clf__C: 0.1
clf__dual: True


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [30]:
cv = CountVectorizer(stop_words='english', ngram_range=(1,1))
test_cv = cv.fit_transform(shuffled_test[:,0])
from scipy.sparse import hstack
test_X = hstack((test_cv, shuffled_test[:,3:6].astype(float)))
test_y = shuffled_test[:,1]

gs_clf = GridSearchCV(text_clf, parameters, cv=5, iid=False, n_jobs=-1)
gs_clf = gs_clf.fit(test_X, test_y)
print(gs_clf.best_score_)
for param_name in sorted(parameters.keys()):
  print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))

0.5831352574985852
clf__alpha: 1


In [27]:
cv = CountVectorizer(stop_words='english', ngram_range=(1,1))
test_cv = cv.fit_transform(shuffled_test[:,0])
from scipy.sparse import hstack
test_X = hstack((test_cv, shuffled_test[:,4:].astype(float)))
test_y = shuffled_test[:,1]

from sklearn.linear_model import SGDClassifier
text_clf_SVM = Pipeline([
    ('clf', SGDClassifier()),
])
from sklearn.model_selection import GridSearchCV
parameters_SVM = {
    'clf__alpha':(1,1e-1,1e-2,1e-3),
    'clf__penalty':('l1','l2'),
}
gs_clf_SVM = GridSearchCV(text_clf_SVM, parameters_SVM, cv=5, iid=False, n_jobs=-1)
gs_clf_SVM = gs_clf_SVM.fit(test_X, test_y)
print(gs_clf_SVM.best_score_)
for param_name in sorted(parameters_SVM.keys()):
  print("%s: %r" % (param_name, gs_clf_SVM.best_params_[param_name]))

0.5971977842945585
clf__alpha: 0.001
clf__penalty: 'l1'


In [28]:
cv = CountVectorizer(stop_words='english', ngram_range=(1,1))
test_cv = cv.fit_transform(shuffled_test[:,0])
from scipy.sparse import hstack
test_X = hstack((test_cv, shuffled_test[:,4:6].astype(float)))
test_y = shuffled_test[:,1]

from sklearn.linear_model import SGDClassifier
text_clf_SVM = Pipeline([
    ('clf', SGDClassifier()),
])
from sklearn.model_selection import GridSearchCV
parameters_SVM = {
    'clf__alpha':(1,1e-1,1e-2,1e-3),
    'clf__penalty':('l1','l2'),
}
gs_clf_SVM = GridSearchCV(text_clf_SVM, parameters_SVM, cv=5, iid=False, n_jobs=-1)
gs_clf_SVM = gs_clf_SVM.fit(test_X, test_y)
print(gs_clf_SVM.best_score_)
for param_name in sorted(parameters_SVM.keys()):
  print("%s: %r" % (param_name, gs_clf_SVM.best_params_[param_name]))

0.6047932637066763
clf__alpha: 0.01
clf__penalty: 'l1'


In [29]:
cv = CountVectorizer(stop_words='english', ngram_range=(1,2))
test_cv = cv.fit_transform(shuffled_test[:,0])
from scipy.sparse import hstack
test_X = hstack((test_cv, shuffled_test[:,4:6].astype(float)))
test_y = shuffled_test[:,1]

from sklearn.linear_model import SGDClassifier
text_clf_SVM = Pipeline([
    ('clf', SGDClassifier()),
])
from sklearn.model_selection import GridSearchCV
parameters_SVM = {
    'clf__alpha':(1,1e-1,1e-2,1e-3),
    'clf__penalty':('l1','l2'),
}
gs_clf_SVM = GridSearchCV(text_clf_SVM, parameters_SVM, cv=5, iid=False, n_jobs=-1)
gs_clf_SVM = gs_clf_SVM.fit(test_X, test_y)
print(gs_clf_SVM.best_score_)
for param_name in sorted(parameters_SVM.keys()):
  print("%s: %r" % (param_name, gs_clf_SVM.best_params_[param_name]))

0.5966112740306289
clf__alpha: 0.01
clf__penalty: 'l1'


# 隔离隔离！！！

In [47]:
pipeline_obj = Pipeline([('vect',CountVectorizer()),('tfidf',TfidfTransformer()),('clf',SGDClassifier()),])
print("pipeline:",'\n', [name for name, _ in pipeline_obj.steps],'\n')

parameters = {
    'vect__max_df': (0.5, 0.75),'vect__max_features': (None, 5000, 10000),
    'tfidf__use_idf': (True, False),'tfidf__norm': ('l1', 'l2'),
    'clf__alpha': (0.00001, 0.000001), 'clf__n_iter_no_change': (5,10, 50) }
print("parameters:",'\n',parameters,'\n')

grid_search = GridSearchCV( pipeline_obj, parameters, cv=5, n_jobs = -1,verbose=1 )
#print('grid_search','\n',grid_search,'\n') #输出所有参数名及参数候选值
grid_search.fit(test.post_text,test.label),'\n'#遍历执行候选参数，寻找最优参数

best_parameters = dict(grid_search.best_estimator_.get_params())#get实例中的最优参数
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name])),'\n'#输出最有参数结果
#pipeline_obj.set_params(clf__alpha = 1e-05,clf__n_iter = 50,tfidf__use_idf = True,vect__max_df = 0.5,vect__max_features = None)
#将pipeline_obj实例中的参数重写为最优结果'''
#print(pipeline_obj.named_steps)


#用最优参数训练模型'''
# pipeline_obj.fit(Xtrain,Ytrain)
# pred = pipeline_obj.predict(Xtrain)
# print('\n',metrics.classification_report(Ytrain,pred))
# pred = pipeline_obj.predict(Xtest)
# print('\n',metrics.classification_report(Ytest,pred))

pipeline: 
 ['vect', 'tfidf', 'clf'] 

parameters: 
 {'vect__max_df': (0.5, 0.75), 'vect__max_features': (None, 5000, 10000), 'tfidf__use_idf': (True, False), 'tfidf__norm': ('l1', 'l2'), 'clf__alpha': (1e-05, 1e-06), 'clf__n_iter_no_change': (5, 10, 50)} 

Fitting 5 folds for each of 144 candidates, totalling 720 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   10.4s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:   38.8s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:  1.5min


	clf__alpha: 1e-05
	clf__n_iter_no_change: 5
	tfidf__norm: 'l1'
	tfidf__use_idf: False
	vect__max_df: 0.5
	vect__max_features: 5000


[Parallel(n_jobs=-1)]: Done 720 out of 720 | elapsed:  2.4min finished


In [49]:
grid_search.best_score_

0.5579625292740047

In [0]:
from sklearn.feature_selection import SelectKBest, chi2
X_new = SelectKBest(chi2, k=20).fit_transform(test_X, test_y)

In [56]:
np.shape(test_X),np.shape(X_new)

((1708, 9577), (1708, 20))

In [57]:

text_clf_SVM = Pipeline([
    ('clf', SGDClassifier()),
])
from sklearn.model_selection import GridSearchCV
parameters_SVM = {
    'clf__alpha':(1,1e-1,1e-2,1e-3),
    'clf__penalty':('l1','l2','None'),
    'clf__loss':('hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'),
    'clf__max_iter':(1000,5000,10000,20000),
    
}
gs_clf_SVM = GridSearchCV(text_clf_SVM, parameters_SVM, cv=5, iid=False, n_jobs=-1)
gs_clf_SVM = gs_clf_SVM.fit(X_new, test_y)
print(gs_clf_SVM.best_score_)
for param_name in sorted(parameters_SVM.keys()):
  print("%s: %r" % (param_name, gs_clf_SVM.best_params_[param_name]))

0.6375692407950473
clf__alpha: 0.01
clf__loss: 'hinge'
clf__max_iter: 20000
clf__penalty: 'None'


# !!!当下最高 0.638

In [0]:
cv = CountVectorizer(stop_words='english', ngram_range=(1,1))

# 隔离隔离！！！

## Method 1: Ngrams
combination of unigrams, bigrams and trigrams as features.

In [0]:
topic = "gay rights"
data = data[data.topic=="gay rights"][["post_text","label"]]
data.head()

,post_text,label
1157,I believe it is a privilage for them . If they...,pro
1158,With marriage comes a lot of privileges . Taxe...,pro
1159,I do n't see why people have a problem with th...,pro
1160,[ *I 'm not homosexual but I do agree . ] Why ...,pro
1161,No-one should marry ! ! ! But gay people have ...,pro


In [0]:
np.random.seed(10)
shuffled_data = np.random.permutation(data)
np.shape(shuffled_data)

(1359, 2)

In [0]:
k = 5
test_size = len(shuffled_data)//k
# for i in range(k):
#   test = shuffled_data[i*test_size:(i+1)*test_size]
# index_test = i
test_data = shuffled_data[:test_size]
train_data = shuffled_data[test_size:]
print(np.shape(test_data))

(271, 2)


In [0]:
# Training
from sklearn.feature_extraction.text import CountVectorizer
#t0 = time()
num_features = 100
cv = CountVectorizer(stop_words='english', ngram_range=(1,3), 
                     max_features=num_features)
X_train = cv.fit_transform(train_data[:,0])
X_test = cv.fit_transform(test_data[:,0])

y_train = train_data[:,1]
y_test = test_data[:,1]

In [0]:
from collections import Counter
Counter(y_train)

Counter({'con': 392, 'pro': 696})

In [0]:
feature_names = cv.get_feature_names()

In [0]:
np.shape(feature_names)

(100,)

In [0]:
feature_names[:10]

['10',
 '10 20',
 '10 20 years',
 '10 million',
 '10 million baby',
 '10 years',
 '10 years greater',
 '100',
 '100 effective',
 '100 effective dont']

In [0]:
np.argsort(feature_names)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

In [0]:
np.argsort(feature_names)[-10:]

array([90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

In [0]:
# Author: Peter Prettenhofer <peter.prettenhofer@gmail.com>
#         Olivier Grisel <olivier.grisel@ensta.org>
#         Mathieu Blondel <mathieu@mblondel.org>
#         Lars Buitinck
# License: BSD 3 clause
import logging
import numpy as np
from optparse import OptionParser
import sys
from time import time
import matplotlib.pyplot as plt

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer, CountVectorizer
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import RidgeClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.extmath import density
from sklearn import metrics

In [0]:
target_names = ['pro', 'con']

In [0]:
def trim(s):
    """Trim string to fit on terminal (assuming 80-column display)"""
    return s if len(s) <= 80 else s[:77] + "..."


# #############################################################################
# Benchmark classifiers
def benchmark(clf, print_report=True, print_cm=True, print_top10=True):
    print('_' * 80)
    print("Training: ")
    print(clf)
    t0 = time()
    clf.fit(X_train, y_train)
    train_time = time() - t0
    print("train time: %0.3fs" % train_time)

    t0 = time()
    pred = clf.predict(X_test)
    test_time = time() - t0
    print("test time:  %0.3fs" % test_time)

    score = metrics.accuracy_score(y_test, pred)
    print("accuracy:   %0.3f" % score)

    if hasattr(clf, 'coef_'):
        print("dimensionality: %d" % clf.coef_.shape[1])
        print("density: %f" % density(clf.coef_))

        if print_top10 and feature_names is not None:
            print("top 10 keywords per class:")
            #print(np.shape(clf.coef_[1]),clf.coef_[1])
            #for i, label in enumerate(target_names):
            top10_idx = np.argsort(clf.coef_[0])[-10:]
            top10_wrd = [feature_names[i] for i in top10_idx]
            print(trim(" %s" % ( " ".join(top10_wrd))))
        print()

    if print_report:
        print("classification report:")
        print(metrics.classification_report(y_test, pred,
                                            target_names=target_names))

    if print_cm:
        print("confusion matrix:")
        print(metrics.confusion_matrix(y_test, pred))

    print()
    clf_descr = str(clf).split('(')[0]
    return clf_descr, score, train_time, test_time

In [0]:
benchmark(MultinomialNB(alpha=.01))

________________________________________________________________________________
Training: 
MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True)
train time: 0.006s
test time:  0.000s
accuracy:   0.520
dimensionality: 100
density: 1.000000
top 10 keywords per class:
 saying thing want let reason couples homosexuality man did need

classification report:
              precision    recall  f1-score   support

         pro       0.31      0.31      0.31        95
         con       0.63      0.64      0.63       176

    accuracy                           0.52       271
   macro avg       0.47      0.47      0.47       271
weighted avg       0.52      0.52      0.52       271

confusion matrix:
[[ 29  66]
 [ 64 112]]



('MultinomialNB',
 0.5202952029520295,
 0.005521059036254883,
 0.00022912025451660156)

In [0]:
clf = LinearSVC(penalty='l1', dual=False,tol=1e-3)
clf.fit(X_train, y_train)
# pred = clf.predict(X_test)
# score = metrics.accur
np.shape(clf.coef_)

(1, 100)

In [0]:
np.shape(clf.coef_)

(1, 100)

In [0]:
from sklearn.svm import LinearSVC
benchmark(LinearSVC(penalty='l1', dual=False,tol=1e-3))

________________________________________________________________________________
Training: 
LinearSVC(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l1', random_state=None, tol=0.001,
          verbose=0)
train time: 0.009s
test time:  0.000s
accuracy:   0.557
dimensionality: 100
density: 0.970000
top 10 keywords per class:


IndexError: ignored

## Method2